# YOLO11 Training on Google Colab

This notebook will help you train your YOLO11 model using your Roboflow dataset and then download the trained model for local inference.

GGOGLE COLAB 


In [ ]:
https://colab.research.google.com/drive/1Jw6NDF8G4AsePjlrfwBfGY5NNpIoztRY#scrollTo=train_code


In [ ]:
!pip install ultralytics roboflow PyYAML

## Step 2: Download Dataset from Roboflow

In [ ]:
from roboflow import Roboflow
rf = Roboflow(api_key="WHzX5kSbytxfLmlWAEXI")
project = rf.workspace("jp-icc04").project("train_animal_dectection")
version = project.version(1)
dataset = version.download("yolov11")

## Step 3: Prepare Data Structure & Fix Paths

Since your dataset might be missing a `valid` folder, this script will split 20% of your training data into a validation set and update the `data.yaml` file so YOLO can find the images correctly.

In [ ]:
import os
import yaml
import shutil
import random

dataset_path = dataset.location
train_images_path = os.path.join(dataset_path, 'train', 'images')
train_labels_path = os.path.join(dataset_path, 'train', 'labels')
valid_path = os.path.join(dataset_path, 'valid')

# 1. Create validation folder if it doesn't exist
if not os.path.exists(valid_path):
    print("Validation set missing. Creating one now...")
    os.makedirs(os.path.join(valid_path, 'images'), exist_ok=True)
    os.makedirs(os.path.join(valid_path, 'labels'), exist_ok=True)
    
    images = [f for f in os.listdir(train_images_path) if f.endswith('.jpg')]
    random.shuffle(images)
    
    # Move 20% to validation
    split_idx = int(len(images) * 0.2)
    val_images = images[:split_idx]
    
    for img in val_images:
        label = img.replace('.jpg', '.txt')
        # Move image
        shutil.move(os.path.join(train_images_path, img), os.path.join(valid_path, 'images', img))
        # Move label if it exists
        label_src = os.path.join(train_labels_path, label)
        if os.path.exists(label_src):
            shutil.move(label_src, os.path.join(valid_path, 'labels', label))
    
    print(f"Created validation set with {len(val_images)} images.")
else:
    print("Validation set already exists.")

# 2. Update data.yaml with absolute paths (Colab prefers these)
yaml_path = os.path.join(dataset_path, 'data.yaml')
with open(yaml_path, 'r') as f:
    config = yaml.safe_load(f)

config['train'] = os.path.join(dataset_path, 'train', 'images')
config['val'] = os.path.join(dataset_path, 'valid', 'images')

# Set test path if directory exists, otherwise use val
test_dir = os.path.join(dataset_path, 'test', 'images')
if os.path.exists(test_dir):
    config['test'] = test_dir
elif 'test' in config:
    del config['test']

with open(yaml_path, 'w') as f:
    yaml.dump(config, f)

print("Success: data.yaml paths updated for Colab environment.")

## Step 4: Train YOLO11 Model

Starting the training process. We use `yolo11n.pt` (Nano) which is fast and efficient.

In [ ]:
from ultralytics import YOLO

# Load pre-trained weights
model = YOLO('yolo11n.pt')

# Start training
results = model.train(
    data=os.path.join(dataset.location, 'data.yaml'),
    epochs=50,
    imgsz=640,
    plots=True
)

## Step 5: Download Trained Model

Download the `best.pt` file to use it locally for your project.

In [ ]:
from google.colab import files
import os

# Path to the best weights generated by training
weights_path = 'runs/detect/train/weights/best.pt'

if os.path.exists(weights_path):
    files.download(weights_path)
else:
    print("Error: weights file not found. Did the training finish?")